In [1]:
### 패키지 설치

import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc #garbage collector

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools #웹 시각화 라이브러리인 d3.js 를 이용하여 보다 interactive 하게 그래프를 만들어주는 모듈.
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #TfidVectorizer: Term Frequency-Inverse Document Frequency(단어 빈도- 역문서 빈도)
from sklearn.decomposition import TruncatedSVD #특이값 분해(Singular Value Decomposition, SVD)

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import plotly.graph_objs as go

import time
import random


In [2]:
data = pd.read_excel('2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803.xlsx')
data.columns = ['홍수사상번호', '연', '월', '일', '시간', '유입량', '유역평균강수_1', '강우(A지역)_1', '강우(B지역)_1', '강우(C지역)_1','강우(D지역)_1', '수위(E지역)_1', '수위(D지역)_1',
               '유역평균강수_2', '강우(A지역)_2', '강우(B지역)_2', '강우(C지역)_2','강우(D지역)_2', '수위(E지역)_2', '수위(D지역)_2',
                '유역평균강수_3', '강우(A지역)_3', '강우(B지역)_3', '강우(C지역)_3','강우(D지역)_3', '수위(E지역)_3', '수위(D지역)_3',
                '유역평균강수_4', '강우(A지역)_4', '강우(B지역)_4', '강우(C지역)_4','강우(D지역)_4', '수위(E지역)_4', '수위(D지역)_4',
                '유역평균강수_5', '강우(A지역)_5', '강우(B지역)_5', '강우(C지역)_5','강우(D지역)_5', '수위(E지역)_5', '수위(D지역)_5',
                '유역평균강수_6', '강우(A지역)_6', '강우(B지역)_6', '강우(C지역)_6','강우(D지역)_6', '수위(E지역)_6', '수위(D지역)_6',
               ]

In [3]:
data = data.drop([0])
data = data.astype('float64')

In [4]:
test = data.iloc[2891:,:]
train = data.iloc[:2891,:]

In [5]:
train_x = train.drop(['홍수사상번호', '연','월','일','시간', '유입량'], axis = 1)
train_y = train.유입량

test_x = test.drop(['홍수사상번호', '연','월','일','시간'], axis = 1)

In [6]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [8]:
random.seed(42)
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
i = 0
xgb_models={}
for train_index, valid_index in kf.split(train_x):
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[valid_index]
    xgb = XGBRegressor(n_estimators=10000000, n_jobs=-1)
    xgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_metric = 'rmse',
            early_stopping_rounds=100,
           verbose=100)
    xgb_models[i]=xgb
    i= i+1

[0]	validation_0-rmse:1453.10388	validation_1-rmse:3881.01953
[100]	validation_0-rmse:15.90570	validation_1-rmse:1930.88098
[153]	validation_0-rmse:9.36894	validation_1-rmse:1929.22656
[0]	validation_0-rmse:2041.59729	validation_1-rmse:1907.04334
[100]	validation_0-rmse:21.50754	validation_1-rmse:580.21924
[109]	validation_0-rmse:19.29562	validation_1-rmse:580.30096
[0]	validation_0-rmse:2126.14697	validation_1-rmse:1491.51733
[100]	validation_0-rmse:23.53544	validation_1-rmse:441.41254
[114]	validation_0-rmse:20.00424	validation_1-rmse:440.65622
[0]	validation_0-rmse:2198.26318	validation_1-rmse:957.73639
[100]	validation_0-rmse:27.60594	validation_1-rmse:384.72916
[104]	validation_0-rmse:26.16468	validation_1-rmse:384.53049
[0]	validation_0-rmse:2156.97339	validation_1-rmse:1257.74023
[100]	validation_0-rmse:23.64237	validation_1-rmse:735.83124
[103]	validation_0-rmse:22.72760	validation_1-rmse:735.83380


In [9]:
pred_y = train_y.copy
pred_y = 0
for fold in range(5):
    pred_y += xgb_models[fold].predict(train_x)/5

In [10]:
from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(train_y, pred_y)
np.sqrt(MSE)

405.3682691717847

In [12]:
xgb1 = XGBRegressor(n_estimators=10000000, n_jobs=-1)
xgb1.fit(train_x, train_y, 
         eval_set=[(X_train, y_train)],
         eval_metric = 'rmse',
         early_stopping_rounds=100,
         verbose=100)

[0]	validation_0-rmse:2163.37695
[100]	validation_0-rmse:29.27097
[200]	validation_0-rmse:11.49457
[300]	validation_0-rmse:5.41596
[400]	validation_0-rmse:2.81601
[500]	validation_0-rmse:1.59671
[600]	validation_0-rmse:0.90282
[700]	validation_0-rmse:0.53407
[800]	validation_0-rmse:0.32883
[900]	validation_0-rmse:0.19629
[1000]	validation_0-rmse:0.12744
[1100]	validation_0-rmse:0.07731
[1200]	validation_0-rmse:0.05019
[1300]	validation_0-rmse:0.03195
[1400]	validation_0-rmse:0.02037
[1500]	validation_0-rmse:0.01264
[1600]	validation_0-rmse:0.00864
[1700]	validation_0-rmse:0.00623
[1800]	validation_0-rmse:0.00502
[1900]	validation_0-rmse:0.00439
[2000]	validation_0-rmse:0.00427
[2033]	validation_0-rmse:0.00427


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10000000, n_jobs=-1, num_parallel_tree=1,
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [16]:
pred_y2 = xgb1.predict(train_x)

In [17]:
from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(train_y, pred_y2)
np.sqrt(MSE)

0.03779617196084325